In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('cleveland297.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [5]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [6]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [7]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [8]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [9]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 3s 164ms/step - loss: 0.7662 - accuracy: 0.5700 - val_loss: 0.6584 - val_accuracy: 0.8444
Epoch 2/100
5/5 [==============================] - 0s 33ms/step - loss: 0.5894 - accuracy: 0.7343 - val_loss: 0.6392 - val_accuracy: 0.8444
Epoch 3/100
5/5 [==============================] - 0s 35ms/step - loss: 0.5992 - accuracy: 0.7101 - val_loss: 0.6300 - val_accuracy: 0.8333
Epoch 4/100
5/5 [==============================] - 0s 33ms/step - loss: 0.6071 - accuracy: 0.7198 - val_loss: 0.6231 - val_accuracy: 0.8556
Epoch 5/100
5/5 [==============================] - 0s 35ms/step - loss: 0.5125 - accuracy: 0.7585 - val_loss: 0.6180 - val_accuracy: 0.8222
Epoch 6/100
5/5 [==============================] - 0s 40ms/step - loss: 0.5606 - accuracy: 0.7295 - val_loss: 0.6101 - val_accuracy: 0.8333
Epoch 7/100
5/5 [==============================] - 0s 45ms/step - loss: 0.5153 - accuracy: 0.7488 - val_loss: 0.6089 - val_accuracy: 0.8000
Epoch 8/100
5/5 [==

In [10]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 5ms/step - loss: 0.4907 - accuracy: 0.8333
Test Loss: 0.4907
Test Accuracy: 0.8333


In [11]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 0s 5ms/step
Confusion Matrix:
[[45  4]
 [11 30]]

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        49
           1       0.88      0.73      0.80        41

    accuracy                           0.83        90
   macro avg       0.84      0.83      0.83        90
weighted avg       0.84      0.83      0.83        90



In [12]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [14]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 14s 761ms/step - loss: 0.6806 - accuracy: 0.6329 - val_loss: 0.6866 - val_accuracy: 0.6333
Epoch 2/100
5/5 [==============================] - 0s 73ms/step - loss: 0.6329 - accuracy: 0.7198 - val_loss: 0.6709 - val_accuracy: 0.5667
Epoch 3/100
5/5 [==============================] - 0s 78ms/step - loss: 0.5967 - accuracy: 0.7005 - val_loss: 0.6438 - val_accuracy: 0.5778
Epoch 4/100
5/5 [==============================] - 0s 75ms/step - loss: 0.6081 - accuracy: 0.7053 - val_loss: 0.6342 - val_accuracy: 0.6333
Epoch 5/100
5/5 [==============================] - 0s 50ms/step - loss: 0.5814 - accuracy: 0.6957 - val_loss: 0.6423 - val_accuracy: 0.6667
Epoch 6/100
5/5 [==============================] - 0s 50ms/step - loss: 0.5780 - accuracy: 0.7150 - val_loss: 0.6431 - val_accuracy: 0.6889
Epoch 7/100
5/5 [==============================] - 0s 55ms/step - loss: 0.5795 - accuracy: 0.7150 - val_loss: 0.6417 - val_accuracy: 0.6667
Epoch 8/100
5/5 [=

In [15]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 15ms/step - loss: 0.6237 - accuracy: 0.6667
Test Loss: 0.6237
Test Accuracy: 0.6667


In [16]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 3s 10ms/step
Confusion Matrix:
[[49  0]
 [30 11]]

Classification Report:
              precision    recall  f1-score   support

           0       0.62      1.00      0.77        49
           1       1.00      0.27      0.42        41

    accuracy                           0.67        90
   macro avg       0.81      0.63      0.59        90
weighted avg       0.79      0.67      0.61        90



In [17]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [18]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [19]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 22s 942ms/step - loss: 0.7235 - accuracy: 0.5556 - val_loss: 0.6892 - val_accuracy: 0.5444
Epoch 2/100
5/5 [==============================] - 1s 136ms/step - loss: 0.6064 - accuracy: 0.6908 - val_loss: 0.6857 - val_accuracy: 0.5444
Epoch 3/100
5/5 [==============================] - 1s 151ms/step - loss: 0.5468 - accuracy: 0.7246 - val_loss: 0.6822 - val_accuracy: 0.5444
Epoch 4/100
5/5 [==============================] - 0s 86ms/step - loss: 0.5278 - accuracy: 0.7391 - val_loss: 0.6833 - val_accuracy: 0.5444
Epoch 5/100
5/5 [==============================] - 0s 80ms/step - loss: 0.5291 - accuracy: 0.7198 - val_loss: 0.6838 - val_accuracy: 0.5444
Epoch 6/100
5/5 [==============================] - 0s 75ms/step - loss: 0.4837 - accuracy: 0.7391 - val_loss: 0.6833 - val_accuracy: 0.5444
Epoch 7/100
5/5 [==============================] - 1s 128ms/step - loss: 0.4616 - accuracy: 0.7874 - val_loss: 0.6822 - val_accuracy: 0.5444
Epoch 8/100
5/5

Epoch 59/100
5/5 [==============================] - 0s 80ms/step - loss: 0.3013 - accuracy: 0.8696 - val_loss: 0.4394 - val_accuracy: 0.8111
Epoch 60/100
5/5 [==============================] - 0s 78ms/step - loss: 0.2983 - accuracy: 0.8792 - val_loss: 0.4420 - val_accuracy: 0.8333
Epoch 61/100
5/5 [==============================] - 0s 79ms/step - loss: 0.2880 - accuracy: 0.8792 - val_loss: 0.4230 - val_accuracy: 0.8222
Epoch 62/100
5/5 [==============================] - 0s 78ms/step - loss: 0.3192 - accuracy: 0.8744 - val_loss: 0.3725 - val_accuracy: 0.8556
Epoch 63/100
5/5 [==============================] - 0s 81ms/step - loss: 0.3192 - accuracy: 0.8309 - val_loss: 0.3641 - val_accuracy: 0.8556
Epoch 64/100
5/5 [==============================] - 0s 78ms/step - loss: 0.3102 - accuracy: 0.8357 - val_loss: 0.3754 - val_accuracy: 0.8444
Epoch 65/100
5/5 [==============================] - 0s 91ms/step - loss: 0.2739 - accuracy: 0.8599 - val_loss: 0.3851 - val_accuracy: 0.8222


In [20]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 20ms/step - loss: 0.3420 - accuracy: 0.8556
Test Loss: 0.3420
Test Accuracy: 0.8556


In [21]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 3s 19ms/step
Confusion Matrix:
[[45  4]
 [ 9 32]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.92      0.87        49
           1       0.89      0.78      0.83        41

    accuracy                           0.86        90
   macro avg       0.86      0.85      0.85        90
weighted avg       0.86      0.86      0.85        90



In [22]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



3/3 [==============================] - 0s 20ms/step


In [23]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [24]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8444


In [25]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[47  2]
 [12 29]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.96      0.87        49
           1       0.94      0.71      0.81        41

    accuracy                           0.84        90
   macro avg       0.87      0.83      0.84        90
weighted avg       0.86      0.84      0.84        90

Ensemble Accuracy: 0.8444


In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [27]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [28]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [29]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [30]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [31]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8778


In [32]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[45  4]
 [ 7 34]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.92      0.89        49
           1       0.89      0.83      0.86        41

    accuracy                           0.88        90
   macro avg       0.88      0.87      0.88        90
weighted avg       0.88      0.88      0.88        90

Ensemble Accuracy: 0.8778


In [33]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [34]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8667
Confusion Matrix:
[[45  4]
 [ 8 33]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.88        49
           1       0.89      0.80      0.85        41

    accuracy                           0.87        90
   macro avg       0.87      0.86      0.86        90
weighted avg       0.87      0.87      0.87        90

Ensemble Accuracy: 0.8667
